In [31]:
pip install scikit-learn nltk 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import yaml
from sqlalchemy import create_engine
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import nltk

## Database Connection

In [33]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [34]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [35]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Load table mensajeria_novedadesservicio 

In [36]:
novedad_servicio = pd.read_sql_table("mensajeria_novedadesservicio",url_db)
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


In [41]:
# Obtener los nombres de las características (términos) de la matriz TF-IDF
terms = vectorizer.get_feature_names_out()

# Crear un diccionario para almacenar las palabras clave de cada grupo
top_terms_per_cluster = {}

for i in range(num_clusters):
    # Obtener los índices de los puntos en el grupo i
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    
    # Obtener las filas correspondientes en la matriz TF-IDF
    cluster_tfidf = X[cluster_indices]
    
    # Calcular el promedio de peso de cada término dentro del grupo
    term_weights = np.mean(cluster_tfidf.toarray(), axis=0)
    
    # Obtener los términos con el peso más alto en el grupo
    top_terms_indices = term_weights.argsort()[-5:][::-1]  # Top 5 términos
    top_terms = [terms[index] for index in top_terms_indices]
    
    # Almacenar los términos para el grupo
    top_terms_per_cluster[f"Grupo {i}"] = top_terms

# Mostrar los términos clave de cada grupo
top_terms_per_cluster


{'Grupo 0': ['espera', 'sigo', 'entreguen', 'reciban', 'masivo'],
 'Grupo 1': ['compañero', 'toma', 'servicio', 'entrego', 'continúa'],
 'Grupo 2': ['esperando', 'despacho', 'bodega', 'entrega', 'reciban'],
 'Grupo 3': ['enterado', 'pendiente', 'recoger', 'ok', 'salgo'],
 'Grupo 4': ['servicio', 'camioneta', 'mal', 'programado', 'abandono'],
 'Grupo 5': ['repuesto', 'espera', 'deja', 'asesor', 'señor'],
 'Grupo 6': ['informa', 'entrega', 'encuentra', 'cerrado', 'b66'],
 'Grupo 7': ['rod', 'latín', 'proveedor', 'aquí', 'toy'],
 'Grupo 8': ['recibe', 'b1', 'asunto', 'trasbordo', 'queda'],
 'Grupo 9': ['terminal', 'encomienda', 'enviar', 'alisten', 'espera']}

In [38]:
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


In [39]:
# Diccionario con descripciones de cada grupo
descriptions = {
    "Grupo 0": "Pendiente de Recepción o Confirmación",
    "Grupo 1": "Entrega por Compañero o Reparto Directo",
    "Grupo 2": "Pendiente en Bodega o Almacenamiento",
    "Grupo 3": "Recogida Pendiente o Confirmación en Curso",
    "Grupo 4": "Problemas con el Servicio o Programación",
    "Grupo 5": "Espera de Repuestos o Asistencia Técnica",
    "Grupo 6": "Entrega en Proceso o Lugar Cerrado",
    "Grupo 7": "Pedidos con Proveedor o Ubicación en Espera",
    "Grupo 8": "Recepción Completa o Transbordo",
    "Grupo 9": "Esperando Envío desde Terminal",
    "Grupo 10": "Descripciones sin Sentido"
}

# Convertir el diccionario a un DataFrame
categoria_descripcion = pd.DataFrame([
    {
        "grupo": grupo,
        "terminos_clave": ", ".join(terminos),
        "descripcion": descriptions[grupo]
    }
    for grupo, terminos in top_terms_per_cluster.items()
])
categoria_descripcion

,grupo,terminos_clave,descripcion
0,Grupo 0,"espera, sigo, entreguen, reciban, masivo",Pendiente de Recepción o Confirmación
1,Grupo 1,"compañero, toma, servicio, entrego, continúa",Entrega por Compañero o Reparto Directo
2,Grupo 2,"esperando, despacho, bodega, entrega, reciban",Pendiente en Bodega o Almacenamiento
3,Grupo 3,"enterado, pendiente, recoger, ok, salgo",Recogida Pendiente o Confirmación en Curso
4,Grupo 4,"servicio, camioneta, mal, programado, abandono",Problemas con el Servicio o Programación
5,Grupo 5,"repuesto, espera, deja, asesor, señor",Espera de Repuestos o Asistencia Técnica
6,Grupo 6,"informa, entrega, encuentra, cerrado, b66",Entrega en Proceso o Lugar Cerrado
7,Grupo 7,"rod, latín, proveedor, aquí, toy",Pedidos con Proveedor o Ubicación en Espera
8,Grupo 8,"recibe, b1, asunto, trasbordo, queda",Recepción Completa o Transbordo
9,Grupo 9,"terminal, encomienda, enviar, alisten, espera",Esperando Envío desde Terminal


## Load table into ETL database

In [ ]:
categoria_descripcion.to_sql("dim_descripcion_novedad", etl_conn, index_label="key_descripcion_novedad", if_exists="replace") 